# Overview

This notebook collects data from the [Han et al. 2018](https://www.sciencedirect.com/science/article/pii/S0092867418301168#sec4) mouse cell atlas.

#### Dataset description

- Data can be found at the [GEO Accession GSE108097](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE108097).
- This notebook previously collects data from one sample, ["Brain1"](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2906405), for the purposes of data analysis and exploration.

# 0. Setup

Import packages and specify any important functions here.

In [1]:
# import standard python packages
import pandas as pd
import subprocess, os, dill, sys
import copy

# add the utils and env directories to the path
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

# 1. Download files

#### BioFileDocket
First, make a BioFileDocket for the dataset.

In [2]:
################
# general info #
################

# Specify the name of the species folder in Amazon S3
species = 'Mus_musculus'

# Specify any particular identifying conditions, eg tissue type:
conditions = 'adultbrain'

################
################

sample_BFD = BioFileDocket(species, conditions)

/home/ec2-user/glial-origins/output/Mmus_adultbrain/ already exists
Files will be saved into /home/ec2-user/glial-origins/output/Mmus_adultbrain/


In [3]:
################
# general info #
################

# Specify url and other variables
genome_fasta_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.23_GRCm38.p3/GCF_000001635.23_GRCm38.p3_genomic.fna.gz'
genome_version = 'GRCm38.p3'

annot_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.23_GRCm38.p3/GCF_000001635.23_GRCm38.p3_genomic.gff.gz'
gxc_url = 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2906nnn/GSM2906405/suppl/GSM2906405_Brain1_dge.txt.gz'

###########
# runtime #
###########

protocol = 'curl'

genome_fasta = GenomeFastaFile(
    sampledict = sample_BFD.sampledict,
    version = genome_version,
    url = genome_fasta_url,
    protocol = protocol
)

annot = GenomeGffFile(
    sampledict = sample_BFD.sampledict,
    reference_genome = genome_fasta,
    url = annot_url,
    protocol = protocol
)

gxc = GxcFile(
    sampledict = sample_BFD.sampledict,
    reference_genome = genome_fasta,
    reference_annot = annot,
    url = gxc_url,
    protocol = protocol
)

keyfiles = {
    'annot': annot,
    'genome_fasta': genome_fasta,
    'gxc': gxc
}
sample_BFD.add_keyfiles(keyfiles)

display(vars(sample_BFD))

inferring file name as GCF_000001635.23_GRCm38.p3_genomic.fna.gz
file GCF_000001635.23_GRCm38.p3_genomic.fna.gz already exists at /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.fna.gz
file GCF_000001635.23_GRCm38.p3_genomic.fna.gz unzipped and object renamed to GCF_000001635.23_GRCm38.p3_genomic.fna
inferring file name as GCF_000001635.23_GRCm38.p3_genomic.gff.gz
file GCF_000001635.23_GRCm38.p3_genomic.gff.gz already exists at /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.gff.gz
file GCF_000001635.23_GRCm38.p3_genomic.gff.gz unzipped and object renamed to GCF_000001635.23_GRCm38.p3_genomic.gff
inferring file name as GSM2906405_Brain1_dge.txt.gz
file GSM2906405_Brain1_dge.txt.gz already exists at /home/ec2-user/glial-origins/output/Mmus_adultbrain/GSM2906405_Brain1_dge.txt.gz
file GSM2906405_Brain1_dge.txt.gz unzipped and object renamed to GSM2906405_Brain1_dge.txt


gzip: /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.fna: unknown suffix -- ignored
gzip: /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.gff: unknown suffix -- ignored
gzip: /home/ec2-user/glial-origins/output/Mmus_adultbrain/GSM2906405_Brain1_dge.txt: unknown suffix -- ignored


{'species': 'Mus_musculus',
 'conditions': 'adultbrain',
 'directory': '/home/ec2-user/glial-origins/output/Mmus_adultbrain/',
 'files': {},
 'metadata': <biofile_handling.metadata_object at 0x7f671ab2bc70>,
 'annot': <biofile_handling.GenomeGffFile at 0x7f6669b8ab00>,
 'genome_fasta': <biofile_handling.GenomeFastaFile at 0x7f6669b8aaa0>,
 'gxc': <biofile_handling.GxcFile at 0x7f6669b8af20>}

# 2. Load in the gxc matrix and get gene names

In [6]:
### Exception: Must coerce data gxc matrix into common format ###
if 'coerced' not in sample_BFD.gxc.path:
    genes_matrix = pd.read_csv(sample_BFD.gxc.path, sep = ' ')
    
    genes_matrix.insert(0, 'gene_name', genes_matrix.index)
    genes_matrix.reset_index(inplace = True, drop = True)
    display(genes_matrix)

    new_matrix = copy.deepcopy(sample_BFD.gxc)
    new_matrix.filename = sample_BFD.gxc.filename.replace('.txt', '_coerced.txt')

    genes_matrix.to_csv(new_matrix.path, index = None, sep = '\t')
    sample_BFD.gxc = new_matrix
#################################################################

genes_matrix = pd.read_csv(sample_BFD.gxc.path, sep = '\t', nrows = 10)
display(genes_matrix)

gxc_genes_list = pd.read_csv(sample_BFD.gxc.path, sep = '\t', usecols=[0], names = ['gene_name'])
display(gxc_genes_list)

sample_BFD.metadata.add('num_cells', len(genes_matrix.columns) - 1)
sample_BFD.metadata.add('num_genes', len(gxc_genes_list))

,gene_name,Brain_1.CCGCTAAATAAATAAGGG,Brain_1.AACGCCGATCTTGCCCTC,Brain_1.ACCTGAAGTTTATCGTAA,Brain_1.CTCGCACTGAAACCGCTA,Brain_1.ATCAACATCTCTTCGGGT,Brain_1.GCGAATAGGGTCTATGTA,Brain_1.CGAGTAAGGGTCTAGTCG,Brain_1.ATCTCTTCGTAAGTTGCC,Brain_1.AACGCCTAAGGGCTCGCA,...,Brain_1.TTGGACGCCTAGGAGATC,Brain_1.TTAACTAAAGTTTATGTA,Brain_1.GTCCCGGGACATAGGACT,Brain_1.CCGCTAGGGTTTGCTCAA,Brain_1.TGATCAGCTGTGTCAAAG,Brain_1.TCACTTGAATTATGAAGC,Brain_1.AAGTACGCTGTGTATGTA,Brain_1.CCTAGATAGAGAATTTGC,Brain_1.CATCCCATTTGCGGCTGC,Brain_1.CAAAGTGGGTTTAGCGAG
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610007P14Rik,0,3,1,2,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0610009B22Rik,0,3,0,2,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0610009E02Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0610009L18Rik,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0610009O20Rik,0,1,0,1,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0610010F05Rik,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0610010K14Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0610012G03Rik,1,2,0,1,1,2,0,1,2,...,0,0,0,0,0,0,0,0,0,0
9,0610025J13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,gene_name
0,gene_name
1,0610005C13Rik
2,0610007P14Rik
3,0610009B22Rik
4,0610009E02Rik
...,...
20161,n-R5s28
20162,n-R5s37
20163,n-R5s52
20164,n-R5s88


# 3. Get mapping identifiers

In [9]:
# load in the original GFF-based annotation
models = pd.read_csv(sample_BFD.annot.path, skiprows = 7, header = None, sep = '\t', comment = '#')
display(models)

attributes_column = 8

# Check the structure of fields in the GFF additional fields section
display(models[attributes_column][3])

,0,1,2,3,4,5,6,7,8
0,NC_000067.6,RefSeq,region,1,195471971,.,+,.,ID=id0;Dbxref=taxon:10090;Name=1;chromosome=1;...
1,NC_000067.6,BestRefSeq%2CGnomon,gene,3199731,3671742,.,-,.,"ID=gene0;Dbxref=GeneID:497097,MGI:MGI:3528744;..."
2,NC_000067.6,Gnomon,mRNA,3199731,3671742,.,-,.,"ID=rna0;Parent=gene0;Dbxref=GeneID:497097,Genb..."
3,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"ID=id1;Parent=rna0;Dbxref=GeneID:497097,Genban..."
4,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"ID=id2;Parent=rna0;Dbxref=GeneID:497097,Genban..."
...,...,...,...,...,...,...,...,...,...
2284235,NC_005089.1,RefSeq,region,15451,15509,.,+,.,ID=id1169986;Note=ETAS1%3B extended terminatio...
2284236,NC_005089.1,RefSeq,region,15515,15558,.,+,.,ID=id1169987;Note=ETAS2%3B extended terminatio...
2284237,NC_005089.1,RefSeq,region,16035,16058,.,+,.,ID=id1169988;Note=CSB1%3B conserved sequencing...
2284238,NC_005089.1,RefSeq,region,16089,16104,.,+,.,ID=id1169989;Note=CSB2%3B conserved sequencing...


'ID=id1;Parent=rna0;Dbxref=GeneID:497097,Genbank:XM_006495550.2,MGI:MGI:3528744;gbkey=mRNA;gene=Xkr4;product=X Kell blood group precursor related family member 4%2C transcript variant X1;transcript_id=XM_006495550.2'

In [10]:
# Remove any rows with NaNs
models.dropna(inplace = True)

# Extract field and database cross-ref (dbxref) information into columns
models['field_dictionary'] = models[attributes_column].apply(convert_fields_to_dict_gff)
models['gene_name'] = [d.get('gene') for d in models['field_dictionary']]
models['Dbxref'] = [d.get('Dbxref') for d in models['field_dictionary']]
models['dbxref_dict'] = models['Dbxref'].apply(convert_dbxref_to_dict)

display(models)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,Dbxref,dbxref_dict
0,NC_000067.6,RefSeq,region,1,195471971,.,+,.,ID=id0;Dbxref=taxon:10090;Name=1;chromosome=1;...,"{'ID': 'id0', 'Dbxref': 'taxon:10090', 'Name':...",None,taxon:10090,{'taxon': '10090'}
1,NC_000067.6,BestRefSeq%2CGnomon,gene,3199731,3671742,.,-,.,"ID=gene0;Dbxref=GeneID:497097,MGI:MGI:3528744;...","{'ID': 'gene0', 'Dbxref': 'GeneID:497097,MGI:M...",Xkr4,"GeneID:497097,MGI:MGI:3528744","{'GeneID': '497097', 'MGI': '3528744'}"
2,NC_000067.6,Gnomon,mRNA,3199731,3671742,.,-,.,"ID=rna0;Parent=gene0;Dbxref=GeneID:497097,Genb...","{'ID': 'rna0', 'Parent': 'gene0', 'Dbxref': 'G...",Xkr4,"GeneID:497097,Genbank:XM_006495550.2,MGI:MGI:3...","{'GeneID': '497097', 'Genbank': 'XM_006495550...."
3,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"ID=id1;Parent=rna0;Dbxref=GeneID:497097,Genban...","{'ID': 'id1', 'Parent': 'rna0', 'Dbxref': 'Gen...",Xkr4,"GeneID:497097,Genbank:XM_006495550.2,MGI:MGI:3...","{'GeneID': '497097', 'Genbank': 'XM_006495550...."
4,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"ID=id2;Parent=rna0;Dbxref=GeneID:497097,Genban...","{'ID': 'id2', 'Parent': 'rna0', 'Dbxref': 'Gen...",Xkr4,"GeneID:497097,Genbank:XM_006495550.2,MGI:MGI:3...","{'GeneID': '497097', 'Genbank': 'XM_006495550...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2284235,NC_005089.1,RefSeq,region,15451,15509,.,+,.,ID=id1169986;Note=ETAS1%3B extended terminatio...,"{'ID': 'id1169986', 'Note': 'ETAS1%3B extended...",None,None,None
2284236,NC_005089.1,RefSeq,region,15515,15558,.,+,.,ID=id1169987;Note=ETAS2%3B extended terminatio...,"{'ID': 'id1169987', 'Note': 'ETAS2%3B extended...",None,None,None
2284237,NC_005089.1,RefSeq,region,16035,16058,.,+,.,ID=id1169988;Note=CSB1%3B conserved sequencing...,"{'ID': 'id1169988', 'Note': 'CSB1%3B conserved...",None,None,None
2284238,NC_005089.1,RefSeq,region,16089,16104,.,+,.,ID=id1169989;Note=CSB2%3B conserved sequencing...,"{'ID': 'id1169989', 'Note': 'CSB2%3B conserved...",None,None,None


# 4. Extract gene IDs for mapping to UniprotKB
Specify which set of identifiers will be use to query the [Uniprot ID Mapping Tool](https://www.uniprot.org/id-mapping) via API.

If using an identifier from the `dbxref_dict`, specify the name via string in the `dbxref_datafield` variable.

In [11]:
dbxref_datafield = 'MGI'

models.dropna(axis = 0, subset = ['dbxref_dict'], inplace = True)
models[dbxref_datafield] = [d.get(dbxref_datafield) for d in models['dbxref_dict']]

models_subset = models[['gene_name', dbxref_datafield]].dropna().drop_duplicates()

### EXCEPTION ###
# added extra info because the MGI prefix contains a colon
models_subset['MGI'] = 'MGI:' + models_subset[dbxref_datafield]

display(models_subset)

,gene_name,MGI
1,Xkr4,MGI:3528744
17,Gm18956,MGI:5011141
36,Gm7341,MGI:3648497
136,Gm6101,MGI:3644986
137,Rp1,MGI:1341105
...,...,...
2284221,ND6,MGI:102495
2284223,TrnE,MGI:102488
2284226,CYTB,MGI:102501
2284228,TrnT,MGI:102473


# 5. Generate gene list file to query Uniprot ID Mapping API
Generate a text file ending in `_ids.txt` for sending to the ID mapping API.

In [12]:
gene_list = models_subset[dbxref_datafield].unique()

genelist_object = GeneListFile(
    sampledict = sample_BFD.sampledict,
    sources = [sample_BFD.annot],
    genes = gene_list,
    identifier = dbxref_datafield
    )

Wrote 40680 gene ids to /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mmus_adultbrain_MGI_ids.txt


# 6. Query Uniprot ID Mapping API
Specify the `from_type` variable based on the Uniprot name of the identifier.  
The table below lists some databases and the `from_type` string that the API accepts for that datatype.  

| datatype | `from_type` string | description |
| ---: | :--- | :--- |
| Mouse Genome Informatics | `MGI` | ID starts with `MGI:` |
| Zebrafish Information Network | `ZFIN` | ID starts with `ZDB-GENE-` |
| Xenbase | `Xenbase` | ID starts with `XB-GENE-` |

__NOTE:__ You may have to run the cell below twice - UniProt sometimes throws an "Resource not found" message on the first query to the database.

In [15]:
from_type = 'MGI'
to_type = 'UniProtKB'

uniprot_idmm_object = genelist_object.get_uniprot_ids(ID_MAPPER_LOC, from_type, to_type)
sample_BFD.add_keyfile('uniprot_idmm', uniprot_idmm_object, overwrite = True)

uniprot_idmm = pd.read_csv(sample_BFD.uniprot_idmm.path, sep = '\t')
display(uniprot_idmm)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  470k    0    52  100  470k     19   176k  0:00:02  0:00:02 --:--:--  176k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24    0    24    0     0     30      0 --:--:-- --:--:-- --:--:--    30


,From,Entry,Entry Name,Reviewed,Protein names,Gene Names,Organism,Length
0,MGI:3528744,Q5GH67,XKR4_MOUSE,reviewed,XK-related protein 4 [Cleaved into: XK-related...,Xkr4 Xrg4,Mus musculus (Mouse),647
1,MGI:3528744,Q5DTU2,Q5DTU2_MOUSE,unreviewed,MKIAA1889 protein,Xkr4 AY534250 mKIAA1889,Mus musculus (Mouse),106
2,MGI:1341105,P56716,RP1_MOUSE,reviewed,Oxygen-regulated protein 1 (Retinitis pigmento...,Rp1 Orp1 Rp1h,Mus musculus (Mouse),2095
3,MGI:1341105,A0A140LHJ6,A0A140LHJ6_MOUSE,unreviewed,Oxygen-regulated protein 1,Rp1,Mus musculus (Mouse),1371
4,MGI:1341105,Q4VA02,Q4VA02_MOUSE,unreviewed,Rp1 protein,Rp1 Rp1h,Mus musculus (Mouse),2096
...,...,...,...,...,...,...,...,...
78484,MGI:102501,P00158,CYB_MOUSE,reviewed,Cytochrome b (Complex III subunit 3) (Complex ...,Mt-Cyb Cob Cytb mt-Cytb Mtcyb,Mus musculus (Mouse),381
78485,MGI:102501,Q36290,Q36290_MOUSE,unreviewed,Cytochrome b,mt-Cytb,Mus musculus (Mouse),366
78486,MGI:102501,Q3V2F9,Q3V2F9_MOUSE,unreviewed,Cytochrome b,mt-Cytb,Mus musculus (Mouse),381
78487,MGI:102501,Q3V2T7,Q3V2T7_MOUSE,unreviewed,Cytochrome b (Complex III subunit 3) (Complex ...,mt-Cytb,Mus musculus (Mouse),94


# 7. Extract results and generate Uniprot IDMM
Generates an idmm that links `gene_name`, the `dbxref_datafield` seleted above, and `uniprot_id` returned by API.

In [17]:
uniprot_idpairs = uniprot_idmm[['From', 'Entry']]
uniprot_idpairs.rename(columns = {'From': dbxref_datafield, 'Entry': 'uniprot_id'}, inplace = True)
display(uniprot_idpairs)

uniprot_output_idmm = models_subset.merge(uniprot_idpairs, on = dbxref_datafield)
display(uniprot_output_idmm)

/tmp/ipykernel_46158/3311337425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_idpairs.rename(columns = {'From': dbxref_datafield, 'Entry': 'uniprot_id'}, inplace = True)


,MGI,uniprot_id
0,MGI:3528744,Q5GH67
1,MGI:3528744,Q5DTU2
2,MGI:1341105,P56716
3,MGI:1341105,A0A140LHJ6
4,MGI:1341105,Q4VA02
...,...,...
78484,MGI:102501,P00158
78485,MGI:102501,Q36290
78486,MGI:102501,Q3V2F9
78487,MGI:102501,Q3V2T7


,gene_name,MGI,uniprot_id
0,Xkr4,MGI:3528744,Q5GH67
1,Xkr4,MGI:3528744,Q5DTU2
2,Rp1,MGI:1341105,P56716
3,Rp1,MGI:1341105,A0A140LHJ6
4,Rp1,MGI:1341105,Q4VA02
...,...,...,...
78484,CYTB,MGI:102501,P00158
78485,CYTB,MGI:102501,Q36290
78486,CYTB,MGI:102501,Q3V2F9
78487,CYTB,MGI:102501,Q3V2T7


In [20]:
# generate a filename and file for the idmm
uniprot_output_idmm_filename = '_'.join([sample_BFD.species_prefix, conditions, 'uniprot-idmm.tsv'])
uniprot_output_idmm_object = IdmmFile(
    filename = uniprot_output_idmm_filename, 
    sampledict = sample_BFD.sampledict, 
    kind = 'uniprot_idmm', 
    sources = [sample_BFD.annot]
)

# save to file and add to the BioFileDocket
uniprot_output_idmm.to_csv(uniprot_output_idmm_object.path, sep = '\t')
sample_BFD.add_keyfile('uniprot_idmm', uniprot_output_idmm_object)

key "uniprot_idmm" already exists, ignoring


# 8. Convert GFF to GTF

In [21]:
# convert the GFF file to GTF using gffread
models_asgtf = sample_BFD.annot.to_gtf(GFFREAD_LOC)

Converted file GCF_000001635.23_GRCm38.p3_genomic.gtf already exists at:
 /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.gtf


In [22]:
# load the newly-generated GTF file as a dataframe
models_asgtf_df = pd.read_csv(models_asgtf.path, skiprows = 0, header = None, sep = '\t')

display(models_asgtf_df)
display(models_asgtf_df[attributes_column][1])

,0,1,2,3,4,5,6,7,8
0,NC_000067.6,Gnomon,transcript,3199731,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
1,NC_000067.6,Gnomon,exon,3199731,3207317,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
2,NC_000067.6,Gnomon,exon,3213439,3216968,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
3,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
4,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
...,...,...,...,...,...,...,...,...,...
2224794,NC_005089.1,RefSeq,CDS,14145,15288,.,+,0,"transcript_id ""gene48832""; gene_name ""CYTB"";"
2224795,NC_005089.1,RefSeq,transcript,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""..."
2224796,NC_005089.1,RefSeq,exon,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""..."
2224797,NC_005089.1,RefSeq,transcript,15356,15422,.,-,.,"transcript_id ""rna111580""; gene_id ""gene48834""..."


'transcript_id "rna0"; gene_id "gene0"; gene_name "Xkr4";'

In [23]:
# Use a custom function to extract useful fields from the additional fields section (column 8)
# Pull from that dict to fill in additional useful columns
models_asgtf_df['field_dictionary'] = models_asgtf_df[attributes_column].apply(convert_fields_to_dict_gtf)
models_asgtf_df['gene_name'] = [d.get('gene_name') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['gene_id'] = [d.get('gene_id') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['transcript_id'] = [d.get('transcript_id') for d in models_asgtf_df['field_dictionary']]

# Remove CDS annotations because they interfere with TransDecoder cDNA generation
models_asgtf_df = models_asgtf_df[models_asgtf_df[2] != 'CDS']
display(models_asgtf_df)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,gene_id,transcript_id
0,NC_000067.6,Gnomon,transcript,3199731,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
1,NC_000067.6,Gnomon,exon,3199731,3207317,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
2,NC_000067.6,Gnomon,exon,3213439,3216968,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
3,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
4,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224793,NC_005089.1,RefSeq,transcript,14145,15288,.,+,.,"transcript_id ""gene48832""; gene_id ""gene48832""...","{'transcript_id': 'gene48832', 'gene_id': 'gen...",CYTB,gene48832,gene48832
2224795,NC_005089.1,RefSeq,transcript,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""...","{'transcript_id': 'rna111579', 'gene_id': 'gen...",TrnT,gene48833,rna111579
2224796,NC_005089.1,RefSeq,exon,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""...","{'transcript_id': 'rna111579', 'gene_id': 'gen...",TrnT,gene48833,rna111579
2224797,NC_005089.1,RefSeq,transcript,15356,15422,.,-,.,"transcript_id ""rna111580""; gene_id ""gene48834""...","{'transcript_id': 'rna111580', 'gene_id': 'gen...",TrnP,gene48834,rna111580


# 9. Generate gtf-idmm
This file maps the `gene_name` to `gene_id` and `transcript_id` fields generated by the conversion from GFF to GTF, which will be needed for downstream processing.

In [24]:
# Extract gene_name, gene_id, and transcript_id fields to generate an ID mapping matrix (idmm)
idmm_df = models_asgtf_df[['gene_name', 'gene_id', 'transcript_id']].drop_duplicates()
idmm_df.dropna(inplace = True)
display(idmm_df)

# generate a filename and file for the idmm
idmm_filename = '_'.join([sample_BFD.species_prefix, conditions, 'gtf-idmm.tsv'])
idmm = IdmmFile(
    filename = idmm_filename, 
    sampledict = sample_BFD.sampledict, 
    kind = 'gtf_idmm', 
    sources = [sample_BFD.annot]
)

# save to file and add to the BioFileDocket
idmm_df.to_csv(idmm.path, sep = '\t')
sample_BFD.add_keyfile('gtf_idmm', idmm)

,gene_name,gene_id,transcript_id
0,Xkr4,gene0,rna0
8,Xkr4,gene0,rna1
15,Xkr4,gene0,rna2
22,Xkr4,gene0,rna3
29,LOC105243853,gene2,rna4
...,...,...,...
2224789,ND6,gene48830,gene48830
2224791,TrnE,gene48831,rna111578
2224793,CYTB,gene48832,gene48832
2224795,TrnT,gene48833,rna111579


# 10. Generate updated gtf
Generated an updated GTF file using transcript_id as the key. For some datasets, transcripts do not consistently get gene names and gene IDs added, which causes Transdecoder to throw errors. This resolves that problem.

In [25]:
models_asgtf_updated_df = models_asgtf_df.merge(idmm_df, on = 'transcript_id')
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_name': x['gene_name_y']}), axis = 1)
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_id': x['gene_id_y']}), axis = 1)
models_asgtf_updated_df['new_fields'] = models_asgtf_updated_df['field_dictionary'].apply(convert_dict_to_fields_gtf)
models_asgtf_updated_df = models_asgtf_updated_df[[0, 1, 2, 3, 4, 5, 6, 7, 'new_fields']]

models_asgtf_updated_filename = models_asgtf.filename.replace('.gtf', '_updated.gtf')
models_asgtf_updated = GenomeGtfFile(
    filename = models_asgtf_updated_filename, 
    sampledict = sample_BFD.sampledict, 
    reference_genome = sample_BFD.genome_fasta
)

models_asgtf_updated_df.to_csv(models_asgtf_updated.path, header = None, index = None, sep = '\t')

# 11. Generate cDNA and peptide files
Using the updated gtf file and genome file, generate cDNA sequence.

Then, using the cDNA sequence, generate peptide sequences using transdecoder.

Expect this step to take some time, probably ~20-30min.

In [26]:
cdna = sample_BFD.genome_fasta.get_transdecoder_cdna_gtf(models_asgtf_updated, TRANSDECODER_LOC)
sample_BFD.add_keyfile('cdna', cdna)

transdecoder_files = sample_BFD.cdna.to_pep_files(TDLONGORF_LOC, TDPREDICT_LOC)
sample_BFD.add_keyfiles(transdecoder_files)

-- Skipping CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/compute_base_probs.pl /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna 0 > /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//base_freqs.dat, checkpoint [/home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir/.__checkpoints_longorfs/base_freqs_file.ok] exists.
-skipping long orf extraction, already completed earlier as per checkpoint: /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir/.__checkpoints_longorfs/TD.longorfs.ok
-- Skipping CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/get_top_longest_fasta_entries.pl /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//longest_orfs.cds 5000 5000 > /home/ec2-user/glial-origins/output/Mmus_adultbrain/M

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.feature.scores.roc


null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.-.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/deplete_feature_noise.pl  --features_plus /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.+.features  --pwm_minus /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.enhanced.feature.scores.roc


null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//start_refinement.enhanced.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/start_codon_refinement.pl --transcripts /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna --gff3_file /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3 --workdir /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir/ > /home/ec2-user/glial-origins/output/Mmus_adultbrain/Mus_musculus_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3.revised_starts.gff3
Refining start codon selections.
-

# 12. Push files to AWS S3

Iteratively moves through the file_set and file_dict variables and populates files into the right place in AWS.

In [27]:
sample_BFD.local_to_s3()

GCF_000001635.23_GRCm38.p3_genomic.gff already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
GCF_000001635.23_GRCm38.p3_genomic.fna already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
GSM2906405_Brain1_dge_coerced.txt already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
Mmus_adultbrain_uniprot-idmm.tsv already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
Mmus_adultbrain_gtf-idmm.tsv already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder.bed already exists in S3 bucket, skipping upload. set overwrite = True to overwrite the existing file.
GCF_000001635.23_GRCm38.p3_g

# 13. Pickle the `sample_BFD` variable for use by the next script

In [28]:
# Generate a .pkl file for the Docket
sample_BFD.pickle()

# Push to S3, optionally overwriting existing file
sample_BFD.push_to_s3(overwrite = False)

upload: ../../output/Mmus_adultbrain/Mmus_adultbrain_BioFileDocket.pkl to s3://arcadia-reference-datasets/glial-origins-pkl/Mmus_adultbrain_BioFileDocket.pkl
